In [1]:
import snscrape.modules.twitter as sntw
from datetime import date, timedelta
import pandas as pd
import tqdm

In [2]:
# create query date list
query_dates = []
start_date = date(2020, 2, 11)
for i in range(365*3+30*3+17):
    iter_date = start_date + timedelta(i)
    query_dates.append(iter_date.strftime("%Y-%m-%d"))
    
print('Query will start from %s to %s' % (query_dates[0], query_dates[-1]))

Query will start from 2020-02-11 to 2023-05-27


In [3]:
def tweet_to_columns(tweet):
    # Tweet based columns
    tweet_has_links = len(tweet.links)>0 if tweet.links is not None else False
    tweet_has_media = len(tweet.media)>0 if tweet.media is not None else False
    tweet_has_mentions = len(tweet.mentionedUsers)>0 if tweet.mentionedUsers is not None else False
    tweet_has_hashtags = len(tweet.hashtags)>1 if tweet.hashtags is not None else False # other than covid
    tweet_columns = [tweet.id, tweet.date.strftime("%Y-%m-%d"), tweet.url, len(tweet.rawContent), tweet.replyCount,
                     tweet.retweetCount, tweet.likeCount, tweet.quoteCount, tweet.lang, tweet_has_links, tweet_has_media,
                     tweet.retweetedTweet is not None, tweet.quotedTweet is not None, tweet.viewCount, tweet.bookmarkCount,
                     tweet_has_hashtags]
    # User based columns
    user = tweet.user
    user_columns = [user.id, user.displayname, user.rawDescription, user.verified, user.created.strftime("%Y-%m-%d"),
                    user.followersCount, user.friendsCount, user.location]
    tweet_columns.extend(user_columns)
    # Add tweet content as last column
    tweet_columns.append(tweet.rawContent)
    # Column names
    col_names = ['id', 'date', 'url', 'lenContent', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'lang',
             'hasLinks', 'hasMedia', 'isRetweeted', 'isQuoted', 'viewCount', 'bookmarkCount', 'hasHashtags',
             'userId', 'displayName', 'userDescription', 'verified', 'createDate', 'followerCount', 'friendCount',
             'userLocation', 'rawContent']
    return {name: field for name, field in zip(col_names, tweet_columns)}

In [4]:
# formattable twitter query
query = "(#covid) lang:en until:%s since:%s"
# intialize the dataframe
tweets_df = pd.DataFrame()
# saved months
saved_month = set([query_dates[0][:-3]])
# iterate date pairs, and query every day
for i in tqdm.tqdm(range(len(query_dates)-1)):
    day_query = query % (query_dates[i+1], query_dates[i])
    # save DataFrame each month
    if query_dates[i][:-3] not in saved_month:
        print('Saving %d tweets at %s.' % (tweets_df.shape[0], query_dates[i-1][:-3]))
        tweets_df.to_csv('%s.csv' % query_dates[i-1][:-3])
        tweets_df = pd.DataFrame()
        saved_month.add(query_dates[i][:-3])
    print('Running twitter query: "%s"' % day_query)
    j = 0
    for j, tweet in enumerate(sntw.TwitterSearchScraper(day_query).get_items()):
        if j >= 256: # hard limit per day
            break
        # Tweet to DataFrame columns
        df_dict = tweet_to_columns(tweet)
        # Append to DataFrame
        tweets_df = pd.concat([tweets_df, pd.DataFrame(df_dict, index=[0])])
        tweets_df = tweets_df.reset_index(drop=True)
    print('Collected %d tweets at %s.' % (j, query_dates[i]))

print('Saving %d tweets at %s.' % (tweets_df.shape[0], query_dates[-1][:-3]))
tweets_df.to_csv('%s.csv' % query_dates[-1][:-3])

  0%|                                                                                         | 0/1201 [00:00<?, ?it/s]

Running twitter query: "(#covid) lang:en until:2020-02-12 since:2020-02-11"


Stopping after 20 empty pages
  0%|                                                                               | 1/1201 [00:18<6:12:51, 18.64s/it]

Collected 254 tweets at 2020-02-11.
Running twitter query: "(#covid) lang:en until:2020-02-13 since:2020-02-12"


  0%|▏                                                                              | 2/1201 [00:31<5:05:29, 15.29s/it]

...

Collected 256 tweets at 2023-05-24.
Running twitter query: "(#covid) lang:en until:2023-05-26 since:2023-05-25"


Unavailable user in card on tweet 1661856815557259264
User 14204245 not found in user refs in card on tweet 1661856815557259264
Unavailable user in card on tweet 1661732575088611330
Unavailable user in card on tweet 1661732575088611330
User 1062915983294300160 not found in user refs in card on tweet 1661732575088611330
User 1062915983294300160 not found in user refs in card on tweet 1661732575088611330
100%|███████████████████████████████████████████████████████████████████████████▉| 1200/1201 [4:32:46<00:14, 14.72s/it]

Collected 256 tweets at 2023-05-25.
Running twitter query: "(#covid) lang:en until:2023-05-27 since:2023-05-26"


Unavailable user in card on tweet 1662172279051042817
Unavailable user in card on tweet 1662172279051042817
User 1062915983294300160 not found in user refs in card on tweet 1662172279051042817
User 1062915983294300160 not found in user refs in card on tweet 1662172279051042817
Unavailable user in card on tweet 1662168127533375488
Unavailable user in card on tweet 1662168127533375488
User 1062915983294300160 not found in user refs in card on tweet 1662168127533375488
User 1062915983294300160 not found in user refs in card on tweet 1662168127533375488
Unavailable user in card on tweet 1662148193629659137
User 218852667 not found in user refs in card on tweet 1662148193629659137
100%|████████████████████████████████████████████████████████████████████████████| 1201/1201 [4:33:01<00:00, 13.64s/it]

Collected 256 tweets at 2023-05-26.
Saving 6508 tweets at 2023-05.


In [5]:
tweets_df.sample(5)

,id,date,url,lenContent,replyCount,retweetCount,likeCount,quoteCount,lang,hasLinks,...,hasHashtags,userId,displayName,userDescription,verified,createDate,followerCount,friendCount,userLocation,rawContent
5546,1661130151403352064,2023-05-23,https://twitter.com/RenzTom/status/16611301514...,282,49,167,445,8,en,False,...,True,1317304122,Tom Renz,"Renz Law (https://t.co/4nm1IA6XmI), https://t....",False,2013-03-30,133888,90,Ohio,Ron DeSantis saw through COVID corruption. Tod...
4266,1659337576652869632,2023-05-18,https://twitter.com/bradboyqld/status/16593375...,128,0,0,0,0,en,False,...,True,175194959,Parody Freedom-Hater Feldspar Gate Bison 💉,MARIE DAWN CONNORS\n17.4.1932 - 1.2.2019\nMy b...,False,2010-08-05,607,114,,"Even without @qldhealth numbers, it's still a ..."
3285,1657865236559327236,2023-05-14,https://twitter.com/Notlikswehttam/status/1657...,147,0,1,1,0,en,False,...,False,1634235076828381191,Matt Skilton,"Family,\nDog,\nEngland,\nMan Utd,\nWoke to som...",False,2023-03-10,63,822,,Well I can honestly say round 4 of #covid suck...
3668,1658177829077659648,2023-05-15,https://twitter.com/Kathy_Edm/status/165817782...,278,1,0,0,0,en,False,...,True,19360482,"Kathy: Mask for good health, Purple ☮️🇨🇦and 🙃",'great spirits have always encountered violent...,False,2009-01-22,1849,2860,"Alberta, Canada",#DANIELLESMITH discussing drug used by addicts...
973,1654182851812761603,2023-05-04,https://twitter.com/jodrell/status/16541828518...,184,0,0,1,0,en,False,...,True,818671,Adam Reynolds,"Home Brewer, DBA, geek, golfer & gamer EULA ht...",False,2007-03-07,534,394,The Shadow knows,Bin dun had my 6th Covid jab at lunchtime toda...
